# FEMA Disaster Cost Forecasting
#### Capstone 2 - Modeling
Michael Garber


* NOTE: Please run notebooks (below) before this one to create data needed.
    * On Github 
        * > [FEMA-DataWrangling.ipynb on Github](https://github.com/mdgarber/FEMADisasterCostForecasting/blob/ef70129c4bf06a38b13c61e1254fdb6a3105486b/femadisastercostforecasting/notebooks/FEMA-DataWrangling.ipynb)
        * > [FEMA-EDA.ipynb on Github](https://github.com/mdgarber/FEMADisasterCostForecasting/blob/ef70129c4bf06a38b13c61e1254fdb6a3105486b/femadisastercostforecasting/notebooks/FEMA-EDA.ipynb)
        * > [FEMA-Modeling.ipynb on Github](URLhere)
    * OR local path
        * > /FEMADisasterCostForecasting/femadisastercostforecasting/notebooks/FEMA-DataWrangling.ipynb
        * > /FEMADisasterCostForecasting/femadisastercostforecasting/notebooks/FEMA-EDA.ipynb
        * > /FEMADisasterCostForecasting/femadisastercostforecasting/notebooks/FEMA-Modeling.ipynb



#### Pre-processing and Training Data Development High-Level Steps
1. Creating dummy features
2. Scale standardization
3. Split data into training and testing subsets

Goal: Create a cleaned development dataset you can use to complete the
modeling step of your project.